In [1]:
import cv2
import dlib
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from scipy.spatial import distance as dist
from imutils import face_utils

In [2]:
from importlib.metadata import version
version('tensorflow')

'2.6.0'

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

In [4]:
# Use the code from Chukwudi's notebook... save the plt.show() from the testing set as images for this notebook training set
# Eyes: 0 is closed, 1 is open.
# It dataset is ~65% open, 35% closed.
eyes_df = pd.read_csv('Feature1_labels.csv', header=0)
eyes_df.groupby('Eyes')['Eyes'].count()

Eyes
0     727
1    1364
Name: Eyes, dtype: int64

In [5]:
eyes_df

,VersionInfo,BaseName,Name,Length,FullName,Eyes
0,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa0,aaaaaaaaaa0.png,60894,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
1,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa1,aaaaaaaaaa1.png,60992,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
2,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa10,aaaaaaaaaa10.png,64059,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
3,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa100,aaaaaaaaaa100.png,58726,C:\Users\Michael\Desktop\Michael All\VS_Projec...,0
4,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa101,aaaaaaaaaa101.png,58709,C:\Users\Michael\Desktop\Michael All\VS_Projec...,0
...,...,...,...,...,...,...
2086,File: C:\Users\Michael\Desktop\Mic...,pxjkzvqomp95,pxjkzvqomp95.png,58975,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
2087,File: C:\Users\Michael\Desktop\Mic...,pxjkzvqomp96,pxjkzvqomp96.png,54257,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
2088,File: C:\Users\Michael\Desktop\Mic...,pxjkzvqomp97,pxjkzvqomp97.png,58539,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
2089,File: C:\Users\Michael\Desktop\Mic...,pxjkzvqomp98,pxjkzvqomp98.png,59206,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1


In [6]:
# Train test split.
X_train, X_test, y_train, y_test = train_test_split(
 eyes_df['Name'],
 eyes_df['Eyes'],
 test_size=0.2,
 random_state=180,
 shuffle=True)
y_train.groupby(y_train).count()

Eyes
0     571
1    1101
Name: Eyes, dtype: int64

In [7]:
# Use blazeface to detect face + landmarks
print(X_train.iloc[0])
print(len(X_train))

jzmzdispyo45.png
1672


In [8]:
for i in X_train:
    print(i)

jzmzdispyo45.png
mkmgcxaztt5.png
novarhxpbj78.png
adohdulfwb87.png
cccccccccc44.png
adohdulfwb24.png
aayfryxljh130.png
eeeeeeeeee72.png
cccccccccc111.png
novarhxpbj66.png
mkmgcxaztt96.png
mkmgcxaztt9.png
aaaaaaaaaa57.png
aaaaaaaaaa3.png
novarhxpbj36.png
novarhxpbj118.png
jzmzdispyo53.png
mkmgcxaztt83.png
eeeeeeeeee18.png
eeeeeeeeee45.png
bbbbbbbbbb70.png
dpevefkefv10.png
dpevefkefv72.png
aaaaaaaaaa83.png
novarhxpbj29.png
adohdulfwb75.png
mkmgcxaztt0.png
aayfryxljh22.png
dddddddddd57.png
aaaaaaaaaa124.png
oojxonbgow56.png
aayfryxljh18.png
aaaaaaaaaa99.png
opvqdabdap90.png
opvqdabdap136.png
cccccccccc119.png
mkmgcxaztt92.png
novarhxpbj32.png
bbbbbbbbbb43.png
oojxonbgow12.png
eeeeeeeeee125.png
bbbbbbbbbb19.png
mkmgcxaztt55.png
dddddddddd39.png
opvqdabdap16.png
cccccccccc35.png
bbbbbbbbbb58.png
mkmgcxaztt85.png
aaaaaaaaaa119.png
oojxonbgow3.png
dddddddddd112.png
dddddddddd76.png
adohdulfwb122.png
bbbbbbbbbb104.png
eeeeeeeeee110.png
bbbbbbbbbb63.png
opvqdabdap80.png
dddddddddd35.png
jzmzdis

cccccccccc99.png
mkmgcxaztt111.png
aayfryxljh83.png
novarhxpbj31.png
oojxonbgow81.png
adohdulfwb92.png
dpevefkefv64.png
oojxonbgow76.png
dpevefkefv127.png
dddddddddd51.png
novarhxpbj115.png
oojxonbgow85.png
novarhxpbj142.png
novarhxpbj149.png
aaaaaaaaaa89.png
novarhxpbj136.png
eeeeeeeeee120.png
opvqdabdap31.png
opvqdabdap20.png
eeeeeeeeee0.png
bbbbbbbbbb77.png
jzmzdispyo13.png
bbbbbbbbbb22.png
dddddddddd129.png
jzmzdispyo91.png
dpevefkefv95.png
adohdulfwb148.png
dpevefkefv29.png
jzmzdispyo22.png
jzmzdispyo58.png
dddddddddd56.png
aaaaaaaaaa42.png
cccccccccc39.png
novarhxpbj20.png
jzmzdispyo104.png
eeeeeeeeee58.png
bbbbbbbbbb55.png
opvqdabdap15.png
cccccccccc70.png
bbbbbbbbbb61.png
pxjkzvqomp49.png
dddddddddd102.png
aaaaaaaaaa77.png
pxjkzvqomp103.png
oojxonbgow1.png
jzmzdispyo110.png
bbbbbbbbbb49.png
cccccccccc133.png
aaaaaaaaaa144.png
oojxonbgow5.png
cccccccccc5.png
cccccccccc36.png
aayfryxljh45.png
pxjkzvqomp57.png
bbbbbbbbbb96.png
dddddddddd146.png
adohdulfwb41.png
aayfryxljh50.png
px

aaaaaaaaaa114.png
dpevefkefv18.png
aaaaaaaaaa117.png
pxjkzvqomp123.png
eeeeeeeeee142.png
opvqdabdap126.png
bbbbbbbbbb112.png
adohdulfwb54.png
bbbbbbbbbb111.png
jzmzdispyo70.png
aayfryxljh54.png
adohdulfwb100.png
oojxonbgow136.png
dddddddddd124.png
mkmgcxaztt26.png
mkmgcxaztt45.png
dddddddddd23.png
adohdulfwb77.png
dddddddddd100.png
cccccccccc116.png
adohdulfwb43.png
opvqdabdap45.png
mkmgcxaztt6.png
opvqdabdap86.png
jzmzdispyo92.png
oojxonbgow90.png
aaaaaaaaaa85.png
dddddddddd149.png
novarhxpbj70.png
dddddddddd61.png
mkmgcxaztt100.png
aayfryxljh30.png
jzmzdispyo112.png
dpevefkefv6.png
mkmgcxaztt107.png
eeeeeeeeee69.png
dpevefkefv142.png
adohdulfwb115.png
jzmzdispyo43.png
novarhxpbj101.png
mkmgcxaztt3.png
aaaaaaaaaa120.png
mkmgcxaztt84.png
dpevefkefv67.png
novarhxpbj73.png
mkmgcxaztt86.png
aayfryxljh3.png
jzmzdispyo134.png
oojxonbgow66.png
opvqdabdap133.png
dddddddddd91.png
novarhxpbj14.png
cccccccccc13.png
dddddddddd22.png
adohdulfwb23.png
aaaaaaaaaa95.png
cccccccccc72.png
opvqdabdap124

In [9]:
X_train

1286     jzmzdispyo45.png
1440      mkmgcxaztt5.png
1621     novarhxpbj78.png
436      adohdulfwb87.png
685      cccccccccc44.png
              ...        
855      dddddddddd62.png
771     dddddddddd120.png
1767     oojxonbgow74.png
2077     pxjkzvqomp87.png
1035      dpevefkefv9.png
Name: Name, Length: 1672, dtype: object

In [10]:
# Prepare images.
def images_ready(all_imgs):
    result=[]
    for i in all_imgs:
        # load an image from file
        get_file="Feature1_data/"+str(i)
        image = load_img(get_file, target_size=(432, 288))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        result.append(image)
    return result

X_train=images_ready(X_train)
X_test=images_ready(X_test)

In [11]:
# Transfer learn VGG16 to detect eye blinks... binary classification.
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(432, 288, 3))
# don't train upper layers
for layer in model.layers:
    layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(4, activation='relu')(flat1)
output = Dense(2, activation='sigmoid')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 432, 288, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 432, 288, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 432, 288, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 216, 144, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 216, 144, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 216, 144, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 108, 72, 128)      0     

In [12]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

/home/michael/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [13]:
regular=np.array(X_train)
good_shape=np.squeeze(regular)
print(regular.shape)
print(good_shape.shape)

(1672, 1, 432, 288, 3)
(1672, 432, 288, 3)


In [14]:
y_train = tf.one_hot(y_train, depth=2)

In [15]:
print(good_shape[:100][:][:][:].shape)
print(y_train[:100][:].shape)
good_shape=good_shape[:10][:][:][:]
y_train=y_train[:10][:]

(100, 432, 288, 3)
(100, 2)


In [16]:
sess= tf.compat.v1.Session(config=config)

In [ ]:
model.fit(good_shape, y_train, verbose=0)

In [ ]:
K.clear_session()

In [ ]:
# Preprocess images and get results. Break video into still frames.
np.shape(new_X_test)